In [1]:
# Binary classification using PubMedBERT. This makes use of the open-source simple transformers (https://simpletransformers.ai/) 

### MIMIC-III discharge summary, for ICD-9 level 1 `circ' label where `circ' includes ICD-9 codes between 390-459: diseases of the circulatory system.
### Percentage of occurrence is of `circ'in unique hospital admissions in MIMIC-III is 78.4%. 
### The total number of hospital admissions with a recorded discharge summary is 52,710.

In [1]:
import numpy as np
import pandas as pd
import random
import os
import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import heapq
from tqdm.notebook import tqdm
import math
from sklearn import metrics
from simpletransformers.classification import ClassificationModel

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [2]:
df = pd.read_csv("circ_binary_mimic_dis.csv")

dfnew=df.drop(['HADM_ID'],axis=1)
dfnew.columns = ['text','labels']
dfnew.head()

,text,labels
0,admission date discharge date date of birth se...,1
1,admission date discharge date date of birth se...,0
2,admission date discharge date date of birth se...,1
3,admission date discharge date date of birth se...,1
4,admission date discharge date date of birth se...,0


In [3]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dfnew, test_size=0.2)

del df,dfnew

In [4]:
### max_seq_length can be upto 512.
# Create a ClassificationModel
model = ClassificationModel(
       "bert", 
       "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext", 
        args={"num_train_epochs": 3, "train_batch_size": 1, "overwrite_output_dir": True,  "learning_rate": 9e-6,
        "max_seq_length": 128, "save_eval_checkpoints": False, "save_model_every_epoch": False, "output_dir": "output/"}
)

## lr is set to be 9e-6. See https://simpletransformers.ai/docs/usage/#creating-a-task-specific-model for more details

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Ber

In [ ]:
from sklearn.metrics import classification_report

## Only running for 3 epochs here. 
model.train_model(train)
to_predict = test.text.tolist()
preds, outputs = model.predict(to_predict)
y_true = test.labels
print(classification_report(y_true, preds))